In [1]:
import json
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

from my_utils import id2section, section2id

tqdm.pandas()

In [2]:
MINING_DIR = '1-mining'
PREPROCESSING_DIR = '2-preprocessing'
DATA_DIR = 'lol'

mining_dir = Path(MINING_DIR) / DATA_DIR
preprocessing_dir = Path(PREPROCESSING_DIR) / DATA_DIR
preprocessing_dir.mkdir(parents=True, exist_ok=True)

In [3]:
with open(mining_dir / 'info.json') as f:
    news_infos_by_date = json.load(f)

In [4]:
columns = ['date', 'ranking', 'thumbnail', 'headline', 'lede', 'office', 'view']
rows = []
for date, news_infos in news_infos_by_date.items():
    for rank, info in enumerate(news_infos, 1):
        thumbnail = int(info['thumbnail'] is not None)
        rows.append([date, rank, thumbnail, info['title'], info['subContent'], info['officeName'], info['totalCount']])

In [5]:
outer_df = pd.DataFrame(rows, columns=columns)
outer_df.to_csv(preprocessing_dir / 'news-outer.csv', index=False)
outer_df

,date,ranking,thumbnail,headline,lede,office,view
0,20150801,1,1,[롤챔스 핫매치 리뷰] '불량 학생' 노동현의 수능 대박 스토리,"""탈선했어요!""""수능 만점!"" 2015년 7월 31일, KOO 타이거즈와 kt 롤스...",인벤,198042
1,20150801,2,1,"[롤챔스 섬머] 스베누 소닉붐 뉴클리어 - 사신 인터뷰, 앞으로는 승리만 안겨드리겠다",금일(1일) 롤챔스 섬머 시즌 사상 스베누 소닉붐이 첫 번째 승리를 기록했다. 15...,헝그리앱,8657
2,20150801,3,1,[롤챔스 섬머] 뱅 배준식 인터뷰. 솔로랭크를 올리는데는 멘탈이 중요!,Q. 오늘 승리한 소감은 어떻게 되나?오늘 경기에 오랜만에 배성웅(벵기)형와 이지훈...,헝그리앱,416
3,20150801,4,1,"[롬챔스 섬머] 스베누 소닉붐, 꼴찌의 반격 개시! 그 동안 설움을 설욕한 대망의 ...",롤챔스 섬머 2경기 2세트는 기세를 올린 스베누 소닉붐이 승리를 가져갔다. 이 때문...,헝그리앱,0
4,20150801,5,1,"[롬챔스 섬머] 스베누 소닉붐, 한타 대승 기반으로 2세트 승리",롤챔스 섬머 2경기 2세트가 시작됐다. 1세트때 진에어 그린윙스에게 억눌린 스베누 ...,헝그리앱,0
...,...,...,...,...,...,...,...
51811,20200430,26,1,"[LCK 승강전] 다이브 받아치면 곧 승리! 샌드박스, 서라벌에 1세트 선승",30일 종각 LoL 파크에서 열린 열린 2020 LCK 섬머 스플릿 승강전 최종전 ...,인벤,1166
51812,20200430,27,1,"[롤챔스 승강전] 서라벌-샌드박스, 베스트 라인업으로 맞대결",LCK 2020 서머 최종전에서 대결하는 서라벌 게이밍(위)과 샌드박스 게이밍(사진...,데일리e스포츠,1057
51813,20200430,28,1,"‘일방적인 전투 승리’…샌드박스게이밍, 서라벌게이밍 상대로 1세트 승리",30일 ‘2020 우리은행 LCK Spring Split’ 승강전 최종진출전에서는 ...,엑스포츠뉴스,835
51814,20200430,29,1,"[LCK 승강전] 샌드박스, 절묘한 카운터로 손쉽게 기선 제압",[OSEN=고용준 기자] LCK 최후의 승강전에서 샌드박스가 먼저 웃었다. 다이브 ...,OSEN,490


In [6]:
DATETIME_PATTERN = re.compile(r'(\d{4})\.(\d{2})\.(\d{2})\. (?:오전|오후) (\d{1,2}):(\d{2})')

def parse_datetime(x: str) -> str:
    x = x.replace('기사입력', '').replace('최종수정', '').strip()
    Y, m, d, H, M = [int(x) for x in DATETIME_PATTERN.match(x).groups()]
    pm = '오후' in x
    if pm and H < 12:
        H += 12
    elif not pm and H == 12:
        H -= 12
    return datetime(year=Y, month=m, day=d, hour=H, minute=M).strftime('%Y%m%d %H:%M')

REPORTER_PATTERN_STRING = r'[가-힣]{2,4} ?기자'
EMAIL_PATTERN_STRING = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'

METADATA_PATTERN_1 = re.compile(r'[\[(<][^>)\]]*?%s[^>)\]]*?[>)\]](\s*\.)?' % REPORTER_PATTERN_STRING)
METADATA_PATTERN_2 = re.compile(r'[\[(<][^>)\]]*?%s[^>)\]]*?[>)\]](\s*\.)?' % EMAIL_PATTERN_STRING)
METADATA_PATTERN_3 = re.compile(r'(?<=\.) (?:[^.]*?L.A)?[^.]*?%s.*$' % REPORTER_PATTERN_STRING)
METADATA_PATTERN_4 = re.compile(r'(?<=\.)/? [^.]*?%s.*$' % EMAIL_PATTERN_STRING)
METADATA_PATTERN_5 = re.compile(r'(?<=\.) +Copyrights? ⓒ.*$')
METADATA_PATTERN_6 = re.compile(r'(?<=\.) +<생활경제팀>.*$')
METADATA_PATTERN_7 = re.compile(r'(?<=\.) +스포츠경향 뉴스를.*$')

NODOT_LINE_PATTERN = re.compile(r'(?<!\.|\n)\n')
SPACES_PATTERN = re.compile(r'\s+')

EMAIL_PATTERN = re.compile(EMAIL_PATTERN_STRING)
URL_PATTERN = re.compile(r'(?:\(\s)(?:https?|ftp)://[^\s/$.?#].[^\s)]*?\)?')


def extract_text_from_html(soup: BeautifulSoup, office: str='', remove_metadata: bool=False) -> str:
    # 태그 제거
    blacklist = ['script', 'noscript', 'style', 'em', 'table']
    for tagname in blacklist:
        for x in soup.select(tagname):
            x.decompose()
    
    # 라인별 전후방 공백 제거
    x = soup.text.strip()
    x = '\n'.join(a.strip() for a in x.splitlines())

    # 공백 축소
    x = NODOT_LINE_PATTERN.sub('.\n', x)
    x = SPACES_PATTERN.sub(' ', x)
    
    # 메타데이터 제거
    if remove_metadata:
        x_old = x
        x = METADATA_PATTERN_1.sub('', x)
        x = METADATA_PATTERN_2.sub('', x)
        if office:
            x = re.sub(r'[\[(<][^>)\]]*?%s[^>)\]]*?[>)\]](\s*\.)?' % office, '', x)

        x_tmp = x
        x = METADATA_PATTERN_3.sub('', x)
        x = METADATA_PATTERN_4.sub('', x)
        x = METADATA_PATTERN_5.sub('', x)
        x = METADATA_PATTERN_6.sub('', x)
        x = METADATA_PATTERN_7.sub('', x)
        if x == x_tmp:
            print(office)
            print(x_tmp)
            x += '[UNCHANGED!!!]'
    
    # URL, EMAIL 제거
    x = EMAIL_PATTERN.sub(' ', x)
    x = URL_PATTERN.sub(' ', x)
    return x.strip()


def parse_row_from_news(soup: BeautifulSoup, **kwargs) -> list:
    soup = soup.select_one('#main_content')
    title = soup.select_one('#articleTitle').text.strip()
    pick = int(soup.select_one('.article_header > .head_channel').attrs['style'] == 'display: block;')
    paper = int(soup.select_one('.article_info > .sponsor > .sponsor_newspaper') is not None)
    
    dates = [parse_datetime(x.text) for x in soup.select('.article_info > .sponsor > .t11')]
    date_input = dates[0]
    date_modify = dates[1] if len(dates) == 2 else dates[0]

    body = soup.select_one('#articleBodyContents')
    img = int(body.select_one('img') is not None)
    vod = int(body.select_one('.vod_area') is not None)
    contents = extract_text_from_html(body, **kwargs)
    return [title, pick, paper, date_input, date_modify, img, vod, contents]


def parse_row_from_entertain(soup: BeautifulSoup, **kwargs) -> list:
    soup = soup.select_one('#content')
    title = soup.select_one('.end_tit').text.strip()
    pick = 0
    paper = 0

    dates = [parse_datetime(x.text) for x in soup.select('.article_info > .author > em')]
    date_input = dates[0]
    date_modify = dates[1] if len(dates) == 2 else dates[0]

    body = soup.select_one('#articeBody')
    img = int(body.select_one('img') is not None)
    vod = int(body.select_one('.vod_area') is not None)
    contents = extract_text_from_html(body, **kwargs)
    return [title, pick, paper, date_input, date_modify, img, vod, contents]


def parse_row_from_sports(soup: BeautifulSoup, **kwargs) -> list:
    soup = soup.select_one('#content')
    if soup.select_one('.column_wrap'):
        title = soup.select_one('.default_h > .info_tit').text.strip()
        date_input = parse_datetime(soup.select_one('.default_h > .info_date').text)
        date_modify = date_input
    else:
        title = soup.select_one('.news_headline > .title').text.strip()
        dates = [parse_datetime(x.text) for x in soup.select('.news_headline > .info > span')]
        date_input = dates[0]
        date_modify = dates[1] if len(dates) == 2 else dates[0]

    body = soup.select_one('#newsEndContents')
    img = int(body.select_one('img') is not None)
    vod = int(body.select_one('.vod_area') is not None)
    contents = extract_text_from_html(body, **kwargs)
    return [title, date_input, date_modify, img, vod, contents]

In [7]:
columns = ['title', 'date_input', 'date_modify', 'img', 'vod', 'contents']
new_rows = []
for row in tqdm(outer_df.itertuples(), total=len(outer_df)):
    mining_date_dir = mining_dir / row.date
    read_glob = '%04d-*-read.html' % (row.ranking)
    read_path = next(mining_date_dir.glob(read_glob))

    soup = BeautifulSoup(open(read_path).read(), 'html5lib')
    if soup.select_one('#content > .error_page'):
        new_row = [None for _ in columns]
    else:
        new_row = parse_row_from_sports(soup, office=row.office, remove_metadata=True)
    new_rows.append(new_row)

인벤
조금만 걸어도 땀이 쏟아지는 한여름. 시원한 매미 소리에 귀를 기울이며 아파트 단지를 따라 걷다 보니 야트막한 언덕에 학교가 보입니다. 학교 정문을 바라보니 어딘지 익숙합니다. 맞다. 저기 정문에 '권위 있는 세계 컴퓨터 게임 프로그램 대회'에서 우승한 한 프로게이머를 축하하는 현수막이 걸려 있었습니다. '여기가 '벵 The Jungle God 기'의 모교입니까?. 설레는 마음을 안고 언덕을 올라 동북고등학교에 들어서니 여느 고교와 다를 바 없는 익숙한 풍경이 눈에 띄었습니다. 운동장에서 축구를 즐기는 학생들, 농구 코트를 뛰어다니는 즐거운 모습, 한가로이 앉아 여유를 즐기는 아이들까지. 고교 시절을 떠올리며 추억에 젖을 때 오늘 인터뷰의 주인공 '벵기' 배성웅이 모습을 드러냈습니다. SKT T1의 유니폼을 입고 달려온 배성웅은 조금은 머쓱한 표정으로 반갑게 인사했습니다. ■ 학교로 돌아간 정글신 '벵기' 배성웅 - "현수막을 걸어준 친구들이 정말 고마웠어요". "방학인 줄 몰랐어요. 학생들이 정말 많을 줄 알았는데, 다행이네요.". 오랜만에 인터뷰에 긴장한 모습을 보이는 배성웅이 왠지 귀여웠습니다. 학교를 왔으니 먼저 인사를 드려야겠죠? SKT T1의 최병훈 감독님과 함께 교장, 교감 선생님을 뵙고 인사를 나눴고 배성웅이 등교했던 2학년 교실에서 인터뷰가 진행되었습니다. "인터뷰를 하는 걸 좋아하진 않아요. 이렇게 따로 시간을 내서 하는 건 왠지 부끄러워요.". 그동안의 인터뷰 요청에 대한 뒷이야기를 미안한 듯 전하는 배성웅. 매체와의 개별적인 만남은 오랜만이지만 성심성의껏 열심히 대답을 해줬습니다. 그의 모교 방문에는 여러 가지 재미있는 에피소드들이 많았는데요. 그중 하나로 고등학교 3학년 때 담임 선생님을 만나 뵀을 때, 담임 선생님은 배성웅을 기억하지 못했습니다. 담임 선생님은 배성웅 선수를 못알아보던데요?. "학교 다닐 때 아주 조용한 학생이긴 했어요. 그때와 비교하면 살도 많이 쪘고요. 그래도 담임 선생님이 기억해주실 줄 알았는데…. 조금 서운

In [8]:
inner_df = pd.DataFrame(new_rows, columns=columns)
#inner_df.to_csv(preprocessing_dir / 'news-inner.csv', index=False)
#inner_df

In [9]:
full_df = pd.read_csv(preprocessing_dir / 'news-full.csv')
inner_df.contents = full_df.contents
inner_df.to_csv(preprocessing_dir / 'news-inner-2.csv', index=False)

In [10]:
full_df.date_input = inner_df.date_input
full_df.date_modify = inner_df.date_modify
full_df.to_csv(preprocessing_dir / 'news-full.csv', index=False)

In [ ]:
REPORTER_PATTERN = re.compile(r'[가-힣]{2,4} ?기자')

def remove_office_info(x):
    office_pattern = re.compile(r'\w*?%s\w*' % office)
    x = x.strip()
    x = office_pattern.sub(' ', x)
    x = REPORTER_PATTERN.sub(' ', x)
    x = SPACES_PATTERN.sub(' ', x)
    return x.strip()

In [23]:
inner_df[inner_df.contents.apply(lambda x: '[UNCHANGED!!!]' in x if x is not None else False)].contents.to_csv(preprocessing_dir / 'temp.csv')

In [37]:
inner_df = pd.read_csv(preprocessing_dir / 'news-inner.csv')
for i, c in pd.read_csv(preprocessing_dir / 'temp.csv', index_col=0).itertuples():
    inner_df.at[i, 'contents'] = c

In [43]:
inner_df.contents = inner_df.contents.apply(lambda x: re.sub('  +', ' ', x) if x and x is not np.nan else np.nan)
inner_df.contents = inner_df.contents.apply(lambda x: x.replace('?.', '?').replace('!.', '!') if x is not np.nan else x)

In [49]:
inner_df.to_csv(preprocessing_dir / 'news-inner.csv', index=False)